In this lab, you will make a character-based n-gram model from a large sample of text and use it to estimate the probability another sample.

In [24]:
import pdb # for debugging
import numpy as np # for vector operations and math
import sys
def subsequences(l, n):
    return(zip(*(l[i:] for i in range(n))))

In [5]:
#example use of subsequences
print(subsequences(range(9),1))
print(subsequences(range(9),2))

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,)]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8)]


We start by reading the large sample of text in `en_train.txt`

In [21]:
# VISIBLE
# initialize storage for the model
ngram = {}
for i in range(1,4):
    ngram[i] = {}

In [22]:
#VISIBLE
with open('en_train_ascii.txt','r') as f:
    for x in f:
        words = x.replace('\n','').split(' ')
        characters = [x.lower() for x in list(x.replace('\n',''))]        
        
        #NOT VISIBLE: SOLUTION FOR POPULATING THE DATASET
        for i in range(1,4):                        
            for char_sequence in subsequences(characters, i):                
                if ''.join(char_sequence) in ngram[i]:
                    ngram[i][''.join(char_sequence)] += 1
                else:
                    ngram[i][''.join(char_sequence)] = 1        

                    
ngram[0] = np.sum(ngram[1].values())                

In [33]:
print(float(ngram[3]['str']) / ngram[2]['st'])

0.0674187009782


In [78]:
# VISIBLE
def getProbability(character, context, ngram):
    '''get the conditional probability of the character given the context'''
    model_order  = len(context) + 1
    sequence = ''.join(context + [character])
    numerator = ngram[model_order][sequence]
    if model_order == 1:
        denominator = ngram[0]
    else:    
        denominator = ngram[model_order-1][''.join(context)] 
    return(float(numerator) / float(denominator))
        

In [79]:
print(getProbability('e',[], ngram))
print(getProbability('e',['p'], ngram))

0.0943070810343
0.177743743859


In [ ]:
def sample(n, context_length):
    '''produce a string of length n, using the specified context length'''

In [85]:
# probability of a held-out set?
def computeProbTestSet(filename, order):
    probs_store = [] 
    with open(filename,'r') as f:
        for x in f:
            words = x.replace('\n','').split(' ')
            characters = [x.lower() for x in list(x.replace('\n',''))]        
            sequences = subsequences(characters,order)
            sentence_prob = []
            for sequence in sequences:
    #             print(sequence[2])
    #             print(''.join(list(sequence[0:2])))
    #             pdb.set_trace()
                sentence_prob.append(np.log(getProbability(sequence[order-1], list(sequence)[0:order-1], ngram)))
            probs_store.append(np.sum(sentence_prob))    
    print(np.sum(probs_store))



In [107]:
np.log((1/float(27)))*2879780

-9491285.0899819471

In [86]:
computeProbTestSet('en_test_within_ascii.txt',1)

-8025236.98228


In [87]:
computeProbTestSet('en_test_within_ascii.txt',2)

-6364835.02565


In [88]:
computeProbTestSet('en_test_within_ascii.txt',3)

-4885605.8259


In [195]:
# generate a string per a context order
# this data structure isn't great for this
def generateString(charlength, ngram, order):    
    output = []
    for char_index in range(charlength):  
        #print('New Letter')
        #print(str(max(char_index-(order -1), 0))+' - ' +str(char_index))
        preceding_context = output[max(0, char_index-(order -1)):char_index]
        #print('preceding_context: ' + ''.join(preceding_context))
        
        cds = ngram[len(preceding_context)+1].keys()
        #but not all of these are consistent with the preceding context
        filtered_cds = [x for x in cds if list(x)[0:len(preceding_context)] == preceding_context]
        #print(filtered_cds)
        
        filtered_counts = np.array([ngram[len(preceding_context)+1][x] for x in filtered_cds])
        if len(preceding_context) == 0:
            normalizer = float(ngram[0])
        else:
            normalizer = float(np.sum(filtered_counts))
            
        filtered_probs = filtered_counts / normalizer
        #print(np.sum(filtered_probs))
        
        choice = list(np.random.choice(filtered_cds, p = filtered_probs))[-1]
        #print('Choice: '+choice)
        output.append(choice)
        
    return (''.join(output))


In [197]:
generateString(100, ngram, 1)

'nhh t bbht  twn it o etwhrhraooka haui num e bi  tclstarhihasdm n elnaebjr o c oay  os ea hht d og o'

In [198]:
generateString(100, ngram, 2)

'ofr at per terit hut y ig y haveresi e ft witavede ckndckwhingetieat ai mauser s wotheas thyoffellie'

In [196]:
generateString(100, ngram, 3)
#this should work fine without smoothing

'vere hen oneall the throut las is nowright mpir ritump art wilive hill com sont ifflove you will ric'